In [1]:
DATA_PATH = '../data'
TRAIN_DATA_PATH = f'{DATA_PATH}/train'
TASK_DATA_PATH = f'{DATA_PATH}/test'
TRAIN_FILENAME = f'{DATA_PATH}/train_gt.csv'
TASK_FILENAME = f'{DATA_PATH}/test.csv'

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

df = pd.read_csv(TRAIN_FILENAME, header=None, names=['audio', 'label'])
df['audio'] = TRAIN_DATA_PATH + "/" + df['audio']

x, y = df['audio'], df['label'].to_numpy().reshape(-1, 1)

weight_for_0 = (1 / (len(y) - np.sum(y))) * (len(y) / 2.0)
weight_for_1 = (1 / np.sum(y)) * (len(y) / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

{0: 0.7903573352486982, 1: 1.3610080395794681}


/opt/miniconda3/envs/bank-cnn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import librosa
import numpy as np
from transformers import ASTFeatureExtractor

feature_extractor = ASTFeatureExtractor()

def preprocess_function(audio_path):
    y, sr = librosa.load(audio_path)

    inputs = feature_extractor(y, return_tensors='np', feature_size=-1, sampling_rate=16000) # Probably not 16000
    return inputs['input_values'].reshape(1024, 128, 1)

def pad_each(x, size=1500):
    new_x = []

    for i in tqdm(x):
        if i.shape[0] > size:
            new_x.append(i[:size])
        else:
            new_x.append(np.pad(i, [(0, size-len(i)), (0, 0), (0, 0)]))

    return new_x

/opt/miniconda3/envs/bank-cnn/lib/python3.11/site-packages/transformers/audio_utils.py:297: UserWarning: At least one mel filter has all zero values. The value for `num_mel_filters` (128) may be set too high. Or, the value for `num_frequency_bins` (256) may be set too low.
  warnings.warn(


In [ ]:
# features_list = []

# for audio in tqdm(x):
#     features_list.append(preprocess_function(audio))

# X = np.stack(features_list)

100%|██████████| 8803/8803 [03:25<00:00, 42.75it/s]


: 

In [ ]:
# np.save('../x_bank_cached.npy', X)

: 

In [4]:
X = np.load('../x_bank_cached.npy')

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=52)

print('Относительное количество положительный записей')
print(f'Вся выборка: {sum(y) / len(y)}')
print(f'Обучающая выборка: {sum(y_train) / len(y_train)}')
print(f'Валидационная выборка: {sum(y_test) / len(y_test)}')

Относительное количество положительный записей
Вся выборка: [0.36737476]
Обучающая выборка: [0.3649789]
Валидационная выборка: [0.37296479]


In [6]:
import maximal as ml
from maximal.layers import Attention
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Model

In [7]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv = Sequential([
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D(pool_size=(6, 6)),
      Conv2D(128, (3, 3), activation='relu'),
      MaxPooling2D(pool_size=(6, 6)),
      Conv2D(128, (3, 3), activation='relu'),
    ])
    self.attention = Attention(depth=3)
    self.flatten = Flatten()
    self.denses = Sequential([
      Dense(256, activation='relu'),
      Dense(1, activation='sigmoid')
    ])


  def call(self, x):
    x = self.conv(x)
    x = self.attention(x, x, x)
    print(x.shape)
    x = self.flatten(x)
    x = self.denses(x)
    return x

model = MyModel()

In [8]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.SGD(momentum=0.5, learning_rate=0.01)

custom_callbacks = [
    EarlyStopping(
        monitor='val_f1_score',
        min_delta=1e-5,
        patience=15,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False
    ),
    ModelCheckpoint(
        './saves/cnn_mfcc_checkpoints/checkpoint.weights.h5',
        monitor='val_f1_score',
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode='auto',
        save_freq='epoch'
    )
]

model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy', tf.keras.metrics.F1Score(average='macro',
                                                  threshold=0.5)]
)

hist = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    validation_split=0.2, 
    epochs=200, 
    callbacks=custom_callbacks, 
    verbose=1, 
    shuffle=True, 
    class_weight=class_weight
)

2024-08-05 13:42:10.427769: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-08-05 13:42:10.427802: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-05 13:42:10.427813: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-05 13:42:10.427839: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-05 13:42:10.427864: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200
(None, 26, 1, 3)
(None, 26, 1, 3)


/opt/miniconda3/envs/bank-cnn/lib/python3.11/site-packages/keras/src/backend/tensorflow/nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2024-08-05 13:42:14.677912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


(None, 26, 1, 3)
193/193 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step - accuracy: 0.4766 - f1_score: 0.4500 - loss: 0.6974(None, 26, 1, 3)
193/193 ━━━━━━━━━━━━━━━━━━━━ 116s 588ms/step - accuracy: 0.4768 - f1_score: 0.4497 - loss: 0.6974 - val_accuracy: 0.6270 - val_f1_score: 0.0000e+00 - val_loss: 0.6882
Epoch 2/200
 35/193 ━━━━━━━━━━━━━━━━━━━━ 1:33 592ms/step - accuracy: 0.6059 - f1_score: 0.1671 - loss: 0.6944

In [ ]:
pd.DataFrame(hist.history).plot(figsize=(8,5))
plt.show()

: 

In [ ]:
pred = model.predict(X)
print(np.sum(pred) / len(pred))
model.evaluate(X, y, return_dict=True)

: 

In [ ]:
pred = model.predict(X_test)
print(np.sum(pred) / len(pred))
model.evaluate(X_test, y_test, return_dict=True)

: 

In [ ]:
model.save('saves/cnn_mfcc.keras')

: 

## Task

In [ ]:
stop

: 

In [ ]:
task = pd.read_csv(TASK_FILENAME, header=None, names=['audio', 'label'])
task['audio'] = TASK_DATA_PATH + "/" + task['audio']

task_x = task['audio']

: 

In [ ]:
task_features_list = []

for audio in tqdm(task_x):
    task_features_list.append(preprocess_function(audio))

task_features_list = pad_each(task_features_list)

task_X = np.stack(task_features_list)

: 

In [ ]:
task_y = model.predict(task_X)
task_y = np.round(task_y).astype(int)

: 

In [ ]:
submit = pd.read_csv(TASK_FILENAME, header=None, names=['audio', 'label'])
submit['label'] = task_y
submit.to_csv('submit.csv', header=False, index=False)

: 